# K-mer Analysis for Machine Learning

**Duration**: 30-40 minutes  
**Level**: Intermediate  
**Prerequisites**: Complete [01_getting_started.ipynb](01_getting_started.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. ✅ Extract k-mers for DNABert/DNABERT-2 preprocessing
2. ✅ Generate minimizers for indexing (minimap2-style)
3. ✅ Analyze k-mer spectrum (frequency distribution)
4. ✅ Use parallel extraction for large datasets (2.2× speedup)
5. ✅ Feed k-mers to machine learning models
6. ✅ Understand evidence-based design (Entry 034)

---

## Why K-mers?

K-mers are subsequences of length k extracted from biological sequences. They're fundamental for:

### Machine Learning
- **DNABert/DNABERT-2**: Transformer models for genomics (k=3,4,5,6)
- **Sequence classification**: Species identification, functional prediction
- **Feature extraction**: Convert sequences to numerical representations

### Genomics
- **Indexing**: minimap2, Bowtie (minimizers)
- **Assembly**: De Bruijn graphs (k-mer spectrum)
- **Comparison**: Alignment-free similarity (k-mer sets)

### biometal v1.1.0 K-mer Features

This notebook showcases the **k-mer operations** added in biometal v1.1.0:
- **Evidence-based design** (Entry 034: 1,357 experiments)
- **Scalar-only** (k-mers are data-structure-bound, NEON provides no benefit)
- **Opt-in parallel** (2.2× speedup for large datasets, 4 threads optimal)
- **Streaming architecture** (constant memory)

### Key Finding (Entry 034):
K-mer operations spend 50-60% on **hashing** and 30-40% on **HashMap operations** → **Not compute-bound** → NEON/GPU provide no benefit → Scalar optimal!

In [ ]:
# Import biometal
import biometal
print(f"biometal version: {biometal.__version__}")
print(f"Expected: 1.1.0 or higher (k-mer operations)")

## 1. K-mer Extraction Basics

Extract overlapping k-mers from a sequence. This is the foundation for all k-mer analysis.

### Function:
- `extract_kmers(sequence, k)` - Returns list of k-mers

### Formula:
Number of k-mers = `len(sequence) - k + 1`

### Example:
```
Sequence: ATGCAT (length 6)
k=3 k-mers: ATG, TGC, GCA, CAT (4 k-mers)
Check: 6 - 3 + 1 = 4 ✓
```

In [ ]:
# Basic k-mer extraction
sequence = b"ATGCATGCATGC"

# Try different k values
for k in [3, 4, 5, 6]:
    kmers = biometal.extract_kmers(sequence, k)
    expected_count = len(sequence) - k + 1
    
    print(f"k={k}:")
    print(f"  K-mers: {[kmer.decode() for kmer in kmers]}")
    print(f"  Count: {len(kmers)} (expected: {expected_count})")
    print()

print("✅ Formula: len(sequence) - k + 1")

## 2. K-mers for DNABert Preprocessing

**DNABert** and **DNABERT-2** are transformer models trained on DNA sequences. They expect:
- **K-mer tokenization**: Convert sequences to overlapping k-mers
- **Common k values**: 3, 4, 5, 6 (DNABert paper recommendation)
- **Format**: Space-separated k-mer strings

### Use Cases:
- **Species classification**: Bacterial taxonomy
- **Promoter prediction**: Gene regulation
- **Splice site detection**: Alternative splicing
- **Functional annotation**: Protein function prediction

In [ ]:
# DNABert preprocessing example
import gzip

# Create sample sequences for DNABert
test_sequences = [
    ("sequence1", b"ATGCATGCATGCATGCATGCATGCATGCATGC"),
    ("sequence2", b"GCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGC"),
    ("sequence3", b"AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"),
]

k = 6  # DNABert often uses k=6

print(f"DNABert Preprocessing (k={k})\n")

for seq_id, sequence in test_sequences:
    # Extract k-mers
    kmers = biometal.extract_kmers(sequence, k)
    
    # Convert to space-separated string (DNABert format)
    kmer_string = " ".join(kmer.decode() for kmer in kmers)
    
    print(f"{seq_id}:")
    print(f"  Original: {sequence.decode()[:30]}...")
    print(f"  K-mers:   {kmer_string[:60]}...")
    print(f"  Count:    {len(kmers)} k-mers")
    print()

print("✅ Ready to feed to DNABert tokenizer!")

## 3. Minimizers for Indexing

**Minimizers** are a subset of k-mers used for efficient indexing (minimap2, Bowtie).

### Algorithm:
1. Slide a window of size `w` across sequence
2. In each window, find the lexicographically smallest k-mer
3. Keep only unique minimizers (position changes)

### Benefits:
- **Reduce storage**: ~1/w of all k-mers (10× reduction for w=10)
- **Fast lookup**: Fewer comparisons
- **minimap2 uses this**: Long-read alignment

### Function:
- `extract_minimizers(sequence, k, w)` - Returns list of (position, k-mer) dicts

### Typical Parameters:
- k=15, w=10 (minimap2 default for long reads)
- k=21, w=11 (minimap2 for short reads)

In [ ]:
# Minimizer extraction (minimap2-style)
sequence = b"ATGCATGCATGCATGCATGCATGCATGC"
k = 6   # K-mer size
w = 10  # Window size

# Extract all k-mers (baseline)
all_kmers = biometal.extract_kmers(sequence, k)

# Extract minimizers (subset)
minimizers = biometal.extract_minimizers(sequence, k, w)

print(f"Sequence: {sequence.decode()}")
print(f"\nAll k-mers (k={k}): {len(all_kmers)} k-mers")
print(f"Minimizers (k={k}, w={w}): {len(minimizers)} minimizers")
print(f"Reduction: {len(all_kmers) / len(minimizers):.1f}×\n")

print("Minimizers:")
for m in minimizers:
    kmer_str = m['kmer'].decode()
    print(f"  Position {m['position']:2d}: {kmer_str}")

print(f"\n✅ {len(minimizers)}/{len(all_kmers)} k-mers retained ({100*len(minimizers)/len(all_kmers):.1f}%)")

## 4. K-mer Spectrum Analysis

**K-mer spectrum** = frequency distribution of k-mers across sequences.

### Uses:
- **Genome size estimation**: Peak frequency → coverage → genome size
- **Repeat detection**: High-frequency k-mers = repeats
- **Error correction**: Low-frequency k-mers = likely errors
- **De Bruijn graphs**: Assembly graph construction

### Function:
- `kmer_spectrum(sequences, k)` - Returns dict of k-mer → count

### Analysis:
- **Peak at 1**: Errors or unique regions
- **Peak at ~coverage**: True genomic k-mers
- **High frequency tail**: Repeats, contamination

In [ ]:
# K-mer spectrum analysis
sequences = [
    b"ATGCATGCAT",  # Sequence 1
    b"GCATGCATGC",  # Sequence 2 (overlaps with 1)
    b"ATGCATGCAT",  # Sequence 3 (same as 1)
    b"AAAAAAAAAA",  # Sequence 4 (different)
]

k = 4

# Calculate spectrum
spectrum = biometal.kmer_spectrum(sequences, k)

print(f"K-mer Spectrum (k={k}):\n")
print(f"Total unique k-mers: {len(spectrum)}")
print(f"Total k-mer occurrences: {sum(spectrum.values())}\n")

# Sort by frequency (descending)
sorted_spectrum = sorted(spectrum.items(), key=lambda x: x[1], reverse=True)

print("Top 10 most frequent k-mers:")
for kmer, count in sorted_spectrum[:10]:
    kmer_str = kmer.decode()
    print(f"  {kmer_str}: {count}× {'█' * count}")

# Frequency distribution
freq_dist = {}
for count in spectrum.values():
    freq_dist[count] = freq_dist.get(count, 0) + 1

print(f"\nFrequency distribution:")
for frequency, num_kmers in sorted(freq_dist.items()):
    print(f"  {num_kmers} k-mers appear {frequency}× {'▓' * num_kmers}")

print("\n✅ K-mer spectrum ready for genome analysis!")

## 5. Parallel K-mer Extraction

For **large datasets** (≥1,000 sequences), parallel extraction provides **2.2× speedup**.

### Evidence (Entry 034):
- **Threshold**: 1,000 sequences (auto-detected)
- **Optimal threads**: 4 (validated experimentally)
- **Speedup**: 2.19-2.38× (mean 2.2×)
- **Memory**: Same as scalar (constant per thread)

### When to Use:
- ✅ Large batches (>1,000 sequences)
- ✅ Preprocessing for ML (batch training)
- ❌ Small datasets (overhead > benefit)
- ❌ Real-time processing (latency-sensitive)

### Class:
- `KmerExtractor(parallel=True, threads=4)` - Configurable extractor
- `.will_use_parallel(num_sequences)` - Check if parallel will be used
- `.extract(sequences, k)` - Extract with auto-parallelization

In [ ]:
# Parallel k-mer extraction demonstration
import time

# Generate a large dataset (simulate real preprocessing)
large_dataset = [b"ATGCATGCATGC" * 10 for _ in range(2000)]  # 2000 sequences

k = 6

print(f"Dataset: {len(large_dataset)} sequences, k={k}\n")

# Method 1: Scalar (default)
print("Method 1: Scalar extraction (default)")
start = time.time()
for seq in large_dataset:
    kmers = biometal.extract_kmers(seq, k)
scalar_time = time.time() - start
print(f"  Time: {scalar_time:.3f}s\n")

# Method 2: Parallel extraction
print("Method 2: Parallel extraction (4 threads)")
extractor = biometal.KmerExtractor(parallel=True, threads=4)

# Check if parallel will be used
will_parallel = extractor.will_use_parallel(len(large_dataset))
print(f"  Will use parallel: {will_parallel}")
print(f"  Threshold: 1000 sequences (Entry 034)")

start = time.time()
all_kmers = extractor.extract(large_dataset, k)
parallel_time = time.time() - start
print(f"  Time: {parallel_time:.3f}s\n")

# Results
speedup = scalar_time / parallel_time
print(f"📊 Results:")
print(f"  Scalar:    {scalar_time:.3f}s")
print(f"  Parallel:  {parallel_time:.3f}s")
print(f"  Speedup:   {speedup:.2f}×")
print(f"  Expected:  ~2.2× (Entry 034)")

if speedup >= 2.0:
    print(f"\n✅ Parallel extraction delivering expected speedup!")
else:
    print(f"\n⚠️  Speedup may vary by system (CPU, memory, dataset)")

## 6. Complete ML Preprocessing Pipeline

Let's build a production workflow for DNABert preprocessing:

### Workflow:
1. **Stream reads** from FASTQ (constant memory)
2. **QC filter** (optional, from notebook 02)
3. **Extract k-mers** (parallel for large batches)
4. **Format for DNABert** (space-separated strings)
5. **Batch and yield** (ready for model)

In [ ]:
# Complete ML preprocessing pipeline
import gzip

# Create test FASTQ data
test_fastq = """@read1
ATGCATGCATGCATGCATGCATGCATGCATGCATGCATGCATGCATGC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@read2
GCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@read3
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
"""

with gzip.open("ml_test.fq.gz", "wt") as f:
    f.write(test_fastq)

def dnabert_preprocessing_pipeline(
    fastq_path,
    k=6,
    batch_size=32,
    min_quality=20,
    use_parallel=True
):
    """
    Complete preprocessing pipeline for DNABert.
    
    Args:
        fastq_path: Path to FASTQ file
        k: K-mer size (3-6 typical for DNABert)
        batch_size: Batch size for parallel extraction
        min_quality: Minimum mean quality (QC filter)
        use_parallel: Use parallel extraction for batches
    
    Yields:
        List of k-mer strings ready for DNABert tokenizer
    """
    stream = biometal.FastqStream.from_path(fastq_path)
    
    # Setup parallel extractor if needed
    if use_parallel:
        extractor = biometal.KmerExtractor(parallel=True, threads=4)
    
    batch = []
    
    for record in stream:
        # Step 1: QC filter (optional)
        mean_q = biometal.mean_quality(record.quality)
        if mean_q < min_quality:
            continue
        
        # Add to batch
        batch.append(record.sequence)
        
        # Process batch when full
        if len(batch) >= batch_size:
            # Step 2: Extract k-mers (parallel if large batch)
            if use_parallel:
                all_kmers = extractor.extract(batch, k)
            else:
                all_kmers = [biometal.extract_kmers(seq, k) for seq in batch]
            
            # Step 3: Format for DNABert (space-separated)
            formatted = []
            for kmers in all_kmers:
                kmer_string = " ".join(kmer.decode() for kmer in kmers)
                formatted.append(kmer_string)
            
            yield formatted
            batch = []
    
    # Process remaining sequences
    if batch:
        if use_parallel:
            all_kmers = extractor.extract(batch, k)
        else:
            all_kmers = [biometal.extract_kmers(seq, k) for seq in batch]
        
        formatted = [" ".join(kmer.decode() for kmer in kmers) for kmers in all_kmers]
        yield formatted

# Run pipeline
print("🔬 DNABert Preprocessing Pipeline\n")
print(f"Parameters:")
print(f"  K-mer size: 6")
print(f"  Batch size: 32")
print(f"  Min quality: Q20")
print(f"  Parallel: Yes (4 threads)\n")

for batch_idx, kmer_batch in enumerate(dnabert_preprocessing_pipeline("ml_test.fq.gz"), 1):
    print(f"Batch {batch_idx}: {len(kmer_batch)} sequences")
    for idx, kmer_string in enumerate(kmer_batch, 1):
        print(f"  {idx}. {kmer_string[:60]}...")
    print()

print("✅ Ready to feed to DNABert model!")
print("\nExample PyTorch integration:")
print("""```python
from transformers import AutoTokenizer, AutoModel

# Load DNABert
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNA_bert_6")
model = AutoModel.from_pretrained("zhihan1996/DNA_bert_6")

# Preprocess with biometal (constant memory!)
for kmer_batch in dnabert_preprocessing_pipeline("huge_file.fq.gz"):
    # Tokenize
    inputs = tokenizer(kmer_batch, return_tensors="pt", padding=True)
    
    # Forward pass
    outputs = model(**inputs)
    
    # Memory stays constant (streaming + batching)
```""")

## Key Takeaways

✅ **K-mer Extraction**: `extract_kmers()` for DNABert (k=3-6)  
✅ **Minimizers**: `extract_minimizers()` for indexing (minimap2-style)  
✅ **K-mer Spectrum**: `kmer_spectrum()` for assembly/QC  
✅ **Parallel Extraction**: 2.2× speedup for large datasets (opt-in)  
✅ **Evidence-Based**: Scalar-only optimal (Entry 034)  
✅ **Streaming**: Constant memory for ML preprocessing  

## Evidence-Based Design (Entry 034)

biometal's k-mer operations are based on 1,357 experiments:

| Operation | NEON | Parallel | Recommended |
|-----------|------|----------|-------------|
| **Minimizers** | 1.02-1.26× | 1.08-1.24× | **Scalar** |
| **Spectrum** | 0.95-1.88× | Sometimes slower! | **Scalar** |
| **Extraction** | 0.98-1.12× | **2.19-2.38×** | **Parallel (opt-in)** |

### Why Scalar Optimal?
K-mer operations are **data-structure-bound**, not compute-bound:
- 50-60% time in **hashing** (not SIMD-able)
- 30-40% time in **HashMap operations** (memory access)
- <10% time in actual computation

→ NEON/GPU provide no benefit
→ Parallel helps with batching (2.2× for ≥1000 sequences)

This validates **minimap2's scalar design** and identifies optimization for **DNABert preprocessing**!

## What's Next?

Continue learning with:

**→ [04_sra_streaming.ipynb](04_sra_streaming.ipynb)**
- Stream from NCBI SRA without downloading
- Analyze 5TB datasets with 5 MB memory
- Real E. coli analysis (SRR390728)
- Network streaming architecture

Or revisit:
- **02_quality_control_pipeline.ipynb**: QC before k-mer extraction
- **01_getting_started.ipynb**: Review basics

---

## Exercises

Try these on your own:

1. **Compare k values**: Try k=3,4,5,6 for DNABert - which is best?
2. **Minimizer compression**: Calculate compression ratio for different w values
3. **Spectrum analysis**: Find peaks in k-mer frequency distribution
4. **Parallel scaling**: Test different thread counts (1,2,4,8)
5. **ML integration**: Connect to a real DNABert model

---

## Resources

- **DNABert Paper**: https://academic.oup.com/bioinformatics/article/37/15/2112/6128680
- **minimap2**: https://github.com/lh3/minimap2
- **Evidence Base**: Entry 034 in apple-silicon-bio-bench
- **biometal Docs**: https://docs.rs/biometal

---

**biometal v1.1.0** - K-mer operations & complexity scoring